In [ ]:
# | default_exp routes.dataflow

In [ ]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

from nbdev.showdoc import show_doc

In [ ]:
# | export


@gd.route_function
async def get_dataflow_by_id(
    dataflow_id: int,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    domo_instance = auth.domo_instance

    url = f"https://{domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}"

    return await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### Sample implementation of get_dataflow_by_id


In [ ]:
from pprint import pprint
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_dataflow_by_id(dataflow_id=70, auth=token_auth)

res.response.keys()


all_keys = []
for action in res.response.get("actions"):
    pprint(action)

list(set(all_keys))

{'dataSourceId': '241025d7-3cca-4369-b7c0-b3264277c0e1',
 'executeFlowWhenUpdated': False,
 'gui': {'color': None,
         'colorSource': None,
         'sampleJson': None,
         'x': None,
         'y': None},
 'id': '241025d7-3cca-4369-b7c0-b3264277c0e1',
 'onlyLoadNewVersions': False,
 'pseudoDataSource': False,
 'targetTableName': 'domostats_people',
 'truncateRows': False,
 'truncateTextColumns': False,
 'type': 'LoadFromVault'}
{'dependsOn': ['241025d7-3cca-4369-b7c0-b3264277c0e1'],
 'gui': {'color': None,
         'colorSource': None,
         'sampleJson': None,
         'x': None,
         'y': None},
 'id': 'c176aa53-122a-4a2e-8b6f-41ca4a72d67c',
 'selectStatement': 'SELECT\n'
                    '`Display Name`,\n'
                    '`User ID`\n'
                    ', @row := @row+1 as id\n'
                    ', @isReset := case when @iter = @rowCounter then 1 else 0 '
                    'end isReset\n'
                    ', @group := @isReset + @group   as rowGro

[]

In [ ]:
# | export
@gd.route_function
async def get_dataflow_execution_history(
    dataflow_id: int,
    auth: dmda.DomoAuth,
    maximum: int = None,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=1,
    debug_loop: bool = False,
    debug_api: bool = False,
):
    limit = 100

    url = f"https://{auth.domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}/executions"

    def arr_fn(res):
        return res.response

    res = await gd.looper(
        auth=auth,
        session=session,
        url=url,
        loop_until_end=True if not maximum else False,
        method="GET",
        offset_params_in_body=False,
        offset_params={"offset": "offset", "limit": "limit"},
        arr_fn=arr_fn,
        maximum=maximum,
        limit=limit,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        debug_api=debug_api,
        debug_loop=debug_loop,
    )

    return res

In [ ]:
show_doc(get_dataflow_execution_history)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/dataflow.py#L41){target="_blank" style="float:right; font-size:smaller"}

### get_dataflow_execution_history

>      get_dataflow_execution_history (dataflow_id:int,
>                                      auth:domolibrary.client.DomoAuth.DomoAuth
>                                      , maximum:int=None,
>                                      parent_class:str=None,
>                                      session:httpx.AsyncClient=None,
>                                      debug_num_stacks_to_drop=1,
>                                      debug_loop:bool=False,
>                                      debug_api:bool=False)

#### sample implementation of get_dataflow_execution_history


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_dataflow_execution_history(dataflow_id=70, maximum=2, auth=token_auth)
print(len(res.response))
res.response[0]

2


{'id': 435276,
 'onboardFlowId': 70,
 'previewRows': 0,
 'dapDataFlowExecutionId': '6ada1dc7-2629-4164-a74b-752567a03ca5',
 'beginTime': 1701969246000,
 'lastUpdated': 1701969246000,
 'state': 'CREATED',
 'activationType': 'MANUAL',
 'dataProcessor': 'MYSQL',
 'dataFlowVersion': 262}

In [ ]:
# | export


@gd.route_function
async def get_dataflow_execution_by_id(
    auth: dmda.DomoAuth,
    dataflow_id: int,
    execution_id : int,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}/executions/{execution_id}"

    return await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        debug_api=debug_api,
        session=session,
    )

In [ ]:
show_doc(get_dataflow_execution_by_id)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/dataflow.py#L79){target="_blank" style="float:right; font-size:smaller"}

### get_dataflow_execution_by_id

>      get_dataflow_execution_by_id (auth:domolibrary.client.DomoAuth.DomoAuth,
>                                    dataflow_id:int, execution_id:int,
>                                    debug_api:bool=False,
>                                    debug_num_stacks_to_drop=1,
>                                    parent_class:str=None,
>                                    session:httpx.AsyncClient=None)

#### sample implementation of `get_dataflow_execution_by_id`


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_dataflow_execution_by_id(
    dataflow_id=70, execution_id=433238, auth=token_auth
)

res.response



{'id': 433238,
 'onboardFlowId': 70,
 'previewRows': 0,
 'dapDataFlowExecutionId': '88e96b86-0dcd-4a84-a946-bcb22224ea2f',
 'beginTime': 1701756991000,
 'endTime': 1701757024000,
 'lastUpdated': 1701757024000,
 'failed': False,
 'state': 'SUCCESS',
 'activationType': 'MANUAL',
 'executionEngine': {'platform': 'K8S', 'engine': 'MYSQL_SQL_ENGINE'},
 'actionResults': [{'actionId': '8edc3d78-3586-47a8-9775-c7b146a50d19',
   'type': 'PublishToVault',
   'wasSuccessful': True,
   'beginTime': 1701757021487,
   'endTime': 1701757022754,
   'rowsProcessed': 1816},
  {'actionId': 'c176aa53-122a-4a2e-8b6f-41ca4a72d67c',
   'type': 'GenerateTableAction',
   'wasSuccessful': True,
   'beginTime': 1701757020983,
   'endTime': 1701757021100,
   'rowsProcessed': 909},
  {'actionId': '241025d7-3cca-4369-b7c0-b3264277c0e1',
   'type': 'DataHubManifestLoaderAction',
   'wasSuccessful': True,
   'beginTime': 1701757015894,
   'endTime': 1701757020972,
   'rowsProcessed': 909},
  {'actionId': 'ac9f630d-d8

In [ ]:
# | export


@gd.route_function
async def execute_dataflow(
    auth: dmda.DomoAuth,
    dataflow_id: int,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/dataprocessing/v1/dataflows/{dataflow_id}/executions"

    return await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        debug_api=debug_api,
        session=session,
    )

In [ ]:
show_doc(execute_dataflow)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/routes/dataflow.py#L102){target="_blank" style="float:right; font-size:smaller"}

### execute_dataflow

>      execute_dataflow (auth:domolibrary.client.DomoAuth.DomoAuth,
>                        dataflow_id:int, debug_api:bool=False,
>                        debug_num_stacks_to_drop=1, parent_class:str=None,
>                        session:httpx.AsyncClient=None)

#### sample execute_dataflow


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await execute_dataflow(dataflow_id=70, auth=token_auth)

ResponseGetData(status=200, response={'id': 435276, 'onboardFlowId': 70, 'previewRows': 0, 'dapDataFlowExecutionId': '6ada1dc7-2629-4164-a74b-752567a03ca5', 'beginTime': 1701969245563, 'lastUpdated': 1701969246331, 'state': 'CREATED', 'activationType': 'MANUAL', 'executionEngine': {'platform': 'K8S', 'engine': 'MYSQL_SQL_ENGINE'}, 'dataProcessor': 'MYSQL', 'dataFlowVersion': 262}, is_success=True, parent_class=None)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()